In [1]:
#Just to reset prometheus_client registry

from prometheus_client import REGISTRY

# Unregister all previously registered metrics
for collector in list(REGISTRY._names_to_collectors.values()):
    try:
        REGISTRY.unregister(collector)
    except KeyError:
        pass


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import roc_auc_score

import pandas as pd
from PIL import Image
from torchvision import models
from torch.utils.data import Dataset
import mlflow
import mlflow.pytorch

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune import Tuner, TuneConfig

from prometheus_client import start_http_server, Gauge, Summary

c:\Users\shrey\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-07 16:36:19,865	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-07 16:36:20,172	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
mlflow.set_experiment("chexpert-jupyter")  # name your experiment

train_loss_metric = Gauge("train_loss", "Training loss per epoch")

training_duration = Summary('training_duration_seconds', 'Total training time in seconds')


2025/05/07 16:36:20 INFO mlflow.tracking.fluent: Experiment with name 'chexpert-jupyter' does not exist. Creating a new experiment.


In [4]:
#minio credentials

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"


In [5]:
# -----------------------
# Config
# -----------------------
BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 1e-4
NUM_CLASSES = 14
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_CSV = "sample_train.csv"
TEST_CSV = "sample_test.csv"
IMG_ROOT = "Chexpert-v1.0"  # Current dir includes CheXpert-v1.0/

LABELS = [
    "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", "Lung Lesion",
    "Edema", "Consolidation", "Pneumonia", "Atelectasis", "Pneumothorax",
    "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices", "No Finding"
]

In [6]:
def train_model(config):
    # Override global hyperparameters
    global BATCH_SIZE, LEARNING_RATE
    BATCH_SIZE = config["batch_size"]
    LEARNING_RATE = config["lr"]

    # Your training loop logic should go here
    # This is a placeholder for demonstration purposes.
    # Replace it with the actual training logic and use tune.report to log metrics.
    for epoch in range(2):  # simulate short training
        loss = 0.01 * epoch + config["lr"] * 0.1  # mock loss
        tune.report(loss=loss)

# Define search space
search_space = {
    "lr": tune.loguniform(1e-5, 1e-3),
    "batch_size": tune.choice([4, 8, 16])
}

# Scheduler for early stopping
scheduler = ASHAScheduler(metric="loss", mode="min")

# Tuner setup
tuner = Tuner(
    train_model,
    param_space=search_space,
    tune_config=TuneConfig(
        num_samples=10,
        scheduler=scheduler,
    ),
)

# Start tuning
tuner.fit()


2025-05-07 16:36:47,869	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_f7042_00005
Traceback (most recent call last):
  File "c:\Users\shrey\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "c:\Users\shrey\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\_private\auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "c:\Users\shrey\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\shrey\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\_private\worker.py", line 2782, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "c:\Users\shrey\AppData\Local\Programs\Python\Python39\lib\site-packages\ray\_private\worker.py", line 929, in get_o

ResultGrid<[
  Result(
    error='RayTaskError(TypeError)',
    metrics={},
    path='C:/Users/shrey/ray_results/train_model_2025-05-07_16-36-20/train_model_f7042_00000_0_batch_size=16,lr=0.0000_2025-05-07_16-36-33',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    error='RayTaskError(TypeError)',
    metrics={},
    path='C:/Users/shrey/ray_results/train_model_2025-05-07_16-36-20/train_model_f7042_00001_1_batch_size=16,lr=0.0003_2025-05-07_16-36-34',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    error='RayTaskError(TypeError)',
    metrics={},
    path='C:/Users/shrey/ray_results/train_model_2025-05-07_16-36-20/train_model_f7042_00002_2_batch_size=16,lr=0.0000_2025-05-07_16-36-34',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    error='RayTaskError(TypeError)',
    metrics={},
    path='C:/Users/shrey/ray_results/train_model_2025-05-07_16-36-20/train_model_f7042_00003_3_batch_size=8,lr=0.0001_2025-05-07_16-36-34',
    filesystem='local',
 

In [7]:
# -----------------------
# Dataset
# -----------------------
class CheXpertDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.df[LABELS] = self.df[LABELS].fillna(0).replace(-1, 0)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root_dir, row["Path"])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        labels = torch.tensor(row[LABELS].values.astype("float32"))
        return image, labels


In [8]:
# -----------------------
# Model
# -----------------------
class CheXpertModel(nn.Module):
    def __init__(self, num_classes=14):
        super().__init__()
        base = models.densenet121(pretrained=True)
        in_features = base.classifier.in_features
        base.classifier = nn.Linear(in_features, num_classes)
        self.model = base

    def forward(self, x):
        return self.model(x)

In [9]:
# -----------------------
# AUC Evaluation
# -----------------------
# def compute_auc(y_true, y_pred):
#     try:
#         aucs = [roc_auc_score(y_true[:, i], y_pred[:, i]) for i in range(y_true.shape[1])]
#         return sum(aucs) / len(aucs)
#     except Exception:
#         return None

In [10]:
# -----------------------
# Transforms
# -----------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [11]:
# -----------------------
# Load Data
# -----------------------
train_ds = CheXpertDataset(TRAIN_CSV, IMG_ROOT, transform)
test_ds = CheXpertDataset(TEST_CSV, IMG_ROOT, transform)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [12]:
#prometheus

In [13]:
with training_duration.time():

    with mlflow.start_run():

        # 🔹 Start Prometheus server only ONCE
        start_http_server(8001)

        # 🔹 Log hyperparameters
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("epochs", EPOCHS)
        mlflow.log_param("learning_rate", LEARNING_RATE)

        model = CheXpertModel(NUM_CLASSES).to(DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
        criterion = nn.BCEWithLogitsLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)

        best_loss = float("inf")
        best_model_path = "best_model.pth"

        for epoch in range(EPOCHS):
            model.train()
            total_loss = 0

            for imgs, labels in train_loader:
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)

            # 🔹 Log to MLflow
            print(f"[Epoch {epoch+1}] Train Loss: {avg_loss:.4f}")
            mlflow.log_metric("train_loss", avg_loss, step=epoch)

            # 🔹 Log to Prometheus
            train_loss_metric.set(avg_loss)

            scheduler.step(avg_loss)

            if avg_loss < best_loss:
                best_loss = avg_loss
                torch.save(model.state_dict(), best_model_path)
                print(f"✅ Saved new best model at epoch {epoch+1} (Loss: {avg_loss:.4f})")

        # 🔹 Log final artifacts
        mlflow.log_artifact(best_model_path)
        mlflow.pytorch.log_model(model, "final_model")


KeyboardInterrupt: 

In [ ]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: file:///c:/Users/shaba/Documents/Collage/mlops/project/MedAI-Scalable-Diagnosis-with-Machine-Learning/mlruns


In [ ]:
# -----------------------
# Evaluation
# -----------------------
model.load_state_dict(torch.load(best_model_path))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(DEVICE)
        outputs = torch.sigmoid(model(imgs)).cpu()
        all_preds.append(outputs)
        all_labels.append(labels)

all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

# Convert probabilities to binary predictions (threshold = 0.5)
preds_binary = (all_preds >= 0.5).astype(int)

# Compute accuracy (macro across all labels)
correct = (preds_binary == all_labels).sum()
total = preds_binary.size
accuracy = correct / total

print(f"\n✅ Test Accuracy (avg over all labels): {accuracy:.4f}")


# Show ground truth vs prediction for a few samples
print("\n📊 Ground Truth vs Predictions (first 3 images):")
for i in range(min(6, len(all_preds))):
    print(f"\nImage {i+1}:")
    print("Labels (GT):     ", all_labels[i].astype(int).tolist())
    print("Predictions (bin):", (all_preds[i] >= 0.5).astype(int).tolist())


C:\Users\shaba\AppData\Local\Temp\ipykernel_30112\3007052085.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))



✅ Test Accuracy (avg over all labels): 0.8286

📊 Ground Truth vs Predictions (first 3 images):

Image 1:
Labels (GT):      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Predictions (bin): [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Image 2:
Labels (GT):      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Predictions (bin): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Image 3:
Labels (GT):      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Predictions (bin): [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Image 4:
Labels (GT):      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Predictions (bin): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Image 5:
Labels (GT):      [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Predictions (bin): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
mlflow.end_run()
